# `Практикум по программированию на языке Python`
<br>

## `Занятие 4: Основы ООП: типизация и полиморфизм, классы данных`
<br><br>

### `Мурат Апишев (mel-lain@yandex.ru)`

#### `Москва, 2022`

### `Напоминание: принципы ООП`

- **Абстракция** - выделение важных свойств объекта и игнорирование прочих<br><br>

- **Инкапсуляция** - хранение данных и методов работы с ними внутри одного класса с доступом к данным только через методы<br><br>

- **Наследование** - возможность создания наследников, получающих все свойства родителей с возможностью их переопределения и расширения<br><br>

- **Полиморфизм** - возможность использования объектов разных типов с общим интерфейсом без информации об их внутреннем устройстве

### `Напоминание: класс, объект, интерфейс`

- __Класс__ представляет собой тип данных (как int или str)
- Это способ описания некоторой сущности, её состояния и возможного поведения
- Поведение при этом зависит от состояния и может его изменять<br><br>
- __Объект__ - это конретный представитель класса (как переменная этого типа)
- У объекта своё состояние, изменяемое поведением
- Поведение полностью определяется правилами, описанными в классе<br><br>
- __Интерфейс__ - это класс, описывающий только поведение, без состояния
- Создать объект типа интерфейса невозможно (если есть их поддержка на уровне языка)
- Поведение полностью определяется правилами, описанными в классе
- Вместо этого описываются классы, которые реализуют этот интерфейс и, в то же время, имеют состояние
<br><br>

### `Абстрактные классы`

- Абстрактные классы представляют собой удобное промежуточное состояние между чистым интерфейсом и полноценным классом
- Эмулировать абстрактные классы можно с помощью методов, которые бросают исключение в своей реализации по-умолчанию:

In [20]:
class Abstract:
    def method(self):
        raise NotImplementedError

#Abstract().method() -> NotImplementedError

- Подход плох тем, что возможность создать объект класса всё равно сохраняется и ошибка произойдёт в момент обращения к методу
- Альтернатива - класс `ABC` (на самом деле его метакласс `ABCMeta`) и декораторы `abstractmethod` из библиотеки `abc`:

In [31]:
from abc import ABC, abstractmethod

class Abstract(ABC):
    @abstractmethod
    def my_abstract_method(self):
        ...

#Abstract() -> TypeError: Can't instantiate abstract class Abstract with abstract method my_abstract_method

class Cls(Abstract):
    def my_abstract_method(self):
        ...

Cls()

### `Абстрактные классы`

- Для версий Python ниже 3.8 есть различные версии декораторов (`@abstractstaticmethod`, `@abstractproperty`, `@abstractclassmethod`), на текущий момент язык поддерживает комбинирование декораторов (`@abstractmethod` должен находиться внутри):

In [32]:
class Abstract(ABC):
    @staticmethod
    @abstractmethod
    def my_abstract_staticmethod():
        ...

    @classmethod
    @abstractmethod
    def my_abstract_classmethod(cls):
        ...

    @property
    @abstractmethod
    def my_abstract_property(self):
        ...

    @my_abstract_property.setter
    @abstractmethod
    def my_abstract_property(self, val):
        ...

### `Аннотация типов`

- Последний принцип ООП, который требуется рассмотреть - полиморфизм
- Для обсуждение интерфейсов и полиморфизма нужно детальнее рассмотреть аннотирование типов и статическую проверку типизации
- До появления аннотирования понятие интерфейса в Python не имело особого смысла


- __Виды типизации__ (одна из классификаций):

    1. Утиная (duck-typing)
    2. Номинальная (nominal type system)
    3. Структурная (structural type system)

### `Подход 1: утиная типизация`

- "Если что-то ведёт себя как утка, значит это - утка"
- Эта концепция возникает в языках с динамической типизацией (Python, JavaScript) и означает, что при использовании объекта
    - его конкретный класс не имеет значения
    - важны его атрибуты (поля и методы)
- Т.е. объект принимается без каких-либо проверок, и если он имеет нужные атрибуты - код выполнится корректно, если не имеет - нет

In [18]:
class A:
    def __eq__(self, val):
        return True

# The object with __eq__ method is expected
print(A() == 3)

True


- Пример выше показывает, что утиная типизация при отсутствии контроля типов может привести к неприятным последствиям
- На утиной типизации основан механизм _полиморфизма_ в Python

### `Полиморфизм`

- Полиморфизм позволяет работать с объектами, основываясь только на их интерфейсе, без знания типа
- В C++ требуется, чтобы объекты полиморфных классов имели общего предка
- В общем случае в Python это необязательно, достаточно, чтобы объекты поддерживали один интерфейс (duck-typing)

Опишем для примера два класса геометрических фигур:

In [14]:
class Square:
    def __init__(self, side):
        self.side = side
    
    def area(self):
        return self.side ** 2

class Triangle:
    def __init__(self, a, b, c):
        self.a, self.b, self.c = a, b, c

    def area(self):
        s = (self.a + self.b + self.c) / 2.0
        return (s *(s - self.a) * (s - self.b) * (s - self.c)) ** 0.5

### `Полиморфизм`

Теперь опишем функцию, которая ожидает объекты, умеющие вычислять свою площадь:

In [15]:
def compute_areas(figures):
    for figure in figures:
        print(figure.area(), end=' ')

Можем запускать, не беспокоясь о том, что именно представляют собой входные объекты:

In [16]:
compute_areas([Square(10), Triangle(1, 3, 3)])

100 1.479019945774904 

### `Подход 2: номинальная типизация`

- Совместимость типов определяется через явные декларации в коде (имена типов и иерархия наследования)
- Такой подход используется повсеместно в языках со статической типизацией (C++, Java)
- В случае Python этого можно добиться с помощью статической проверки

In [9]:
class Bird:
    def feed(self) -> None: print('OK')

class Duck(Bird):
    def feed(self) -> None: print('OK')

class Goose:
    def feed(self) -> None: print('OK')

def feed(bird: Bird) -> None:
    bird.feed()

feed(Bird())
feed(Duck())
#feed(Goose()) -> static check error (OK in runtime due to duck typing)

OK
OK


### `Напоминание: аннотации типов`

- Ранее были рассмотрены базовые примеры аннотаций для скалярных и контейнерных (generic) типов:

In [19]:
from typing import Set

def print_scalar(obj: int) -> None:
    print(obj)

def print_set(obj: Set[int]) -> None:
    print(obj)

- Возможности `typing` существенно шире. Например, он
    - вводит новые типы (Union и т.п.)
    - позволяет определять собственные generic-классы


- Но для полноценного использования и изучения сперва нужно добавить статическую проверку типов

### `Модуль MyPy: статическая проверка типов`

- Модуль `mypy` - стандартный инструмент для статической проверки аннотированного кода на Python (ещё есть `pytype` и `pyright`)
- Установка и использование стандартные:

In [1]:
#!pip install mypy

def check_last():
    with open('temp.txt', 'w') as fout:
        fout.write(In[len(In)-2])
    !mypy temp.txt

In [109]:
def func(n: int = 10) -> int:
    return n ** 2

func(2.5)

6.25

In [110]:
check_last()

temp.txt:4: error: Argument 1 to "func" has incompatible type "float"; expected "int"
Found 1 error in 1 file (checked 1 source file)


### `Тип Union`

In [2]:
from typing import Union, List, Set

def func(x: Union[List[int], Set[str]]) -> None:
    ...

func([1, 2, 3])
func({1, 2, 3})
func({'s', 't', 'r'})

In [3]:
#check_last()

/bin/sh: 1: mypy: not found


### `Тип Any`

- Тип `Any` говорит о том, что в этом месте может быть произвольный тип, и проверка кода игнорирует все, связанное с переменной типа `Any`


- Для `Any` верны следующие утверждения:
    - любой объект является объектом типа `Any`
    - любой класс является подклассом типа `Any`
    - `Any` и `object` являются подклассами друг друга


- Несмотря на схожесть, `Any` и `object` - это не одно и то же - тип `object` ограничивает множество операций теми, что допускает `object`, а `Any` допускает всё:

In [ ]:
from typing import Any

def func_any(x: Any) -> None: x.nothing()
def func_object(x: object) -> None: x.nothing()

func_any(None)
func_object(None)

In [114]:
check_last()

temp.txt:4: error: "object" has no attribute "nothing"
Found 1 error in 1 file (checked 1 source file)


### `Тип Optional`

In [115]:
from typing import Optional, List

def func(x: Optional[List[int]]) -> None:
    ...

func([1, 2, 3])
func(None)

In [116]:
check_last()

Success: no issues found in 1 source file


В Python 3.10+ допускается синтаксис с "|" (в то же смысле, что и в TypeScript):

In [ ]:
def func(x: list[int] | None) -> None:
    ...

func([1, 2, 3])
func(None)

In [118]:
check_last()

temp.txt:1: error: X | Y syntax for unions requires Python 3.10
Found 1 error in 1 file (checked 1 source file)


### `Тип Literal`

- Тип `Literal` параметризуется не другим типом, а конкретным значением

In [119]:
from typing import Literal

def func(x: Literal[3], y: Literal['something']) -> None:
    ...

func(3, 'something')
func(4, 'nothing')

In [120]:
check_last()

temp.txt:7: error: Argument 1 to "func" has incompatible type "Literal[4]"; expected "Literal[3]"
temp.txt:7: error: Argument 2 to "func" has incompatible type "Literal['nothing']"; expected "Literal['something']"
Found 2 errors in 1 file (checked 1 source file)


### `Определение собственных generic типов`

- Функция `TypeVar` позволяет получить ссылку на тип переменной, имя которой было подано в качестве параметра
- Ссылка на тип параметра позволяет создавать новые generic типы:

In [121]:
from typing import TypeVar, Generic, List

T = TypeVar('T')

class Stack(Generic[T]):
    def __init__(self) -> None:
        self.items: List[T] = []  

    def push(self, item: T) -> None:
        self.items.append(item)

stack: Stack[int] = Stack()
stack.push(10)
stack.push('10')

In [122]:
check_last()

temp.txt:14: error: Argument 1 to "push" of "Stack" has incompatible type "str"; expected "int"
Found 1 error in 1 file (checked 1 source file)


### `Подход 3: структурная типизация`

- Совместимость типов определяется на основе структуры типов, а не на явных декларациях
- Этот подход аналогичен утиной типизации за исключением того, что проверка является статической, а не динамической, это обеспечивает корректность типов без необходимости явных наследований, что повышает гибкость кода и независимость модулей и классов
- Структурная типизация используется в TypeScript и Go


- Python допускает использование структурной типизации с версии 3.8, для чего требуются аннотирование типов и _протоколы_
- Протокол по сути является интерфейсом, которому объект должен удовлетворять для совместимости типов


- В модуле `typing` есть много стандартных протоколов, например:
    - Mapping
    - Iterable
    - Callable
    - Hashable
    - Reversible
    - ...

### `Примеры использования протоколов`

- Протокол `Callable` требует наличия у реализации интерфейса метода `__call__`
- Протокол `Mapping` требует наличия у реализации интерфейса метода `__getitem__`

In [57]:
from typing import Callable

def func(f: Callable[[int, int], bool]) -> bool:
    return f(1,2)

func(lambda x, y: x == y)

False

In [58]:
from typing import Mapping

def func(m: Mapping[str, int], key: str) -> bool:
    return m[key]

func({'k': 0}, 'k')

0

### `Определение собственного протокола`

- Этот и следующий пример взяты из статьи [https://habr.com/ru/post/557898](https://habr.com/ru/post/557898)

Определим протокол и корректную его реализацию:

In [159]:
from typing import Protocol

class Figure(Protocol):
    name: str

    def calculate_area(self) -> float: pass

    def calculate_perimeter(self) -> float: pass

def show(figure: Figure) -> None:
    print(f'S ({figure.name}) = {figure.calculate_area()}')
    print(f'P ({figure.name}) = {figure.calculate_perimeter()}')

In [160]:
class Square:
    name = 'square'

    def __init__(self, size: float): self.size = size
    def calculate_area(self) -> float: return self.size * self.size
    def calculate_perimeter(self) -> float: return 4 * self.size
        
    def set_color(self, color: str) -> None: self.color = color

show(Square(size=3.14))

S (square) = 9.8596
P (square) = 12.56


### `Определение собственного протокола`

Определим некорректную реализацию и отловим ошибку на этапе статической проверки:

In [ ]:
#from typing import Protocol

#class Figure(Protocol): <class body>

#def show(figure: Figure) -> None: <function body>

class Circle:
    PI = 3.1415926
    name = "Cirsle"

    def __init__(self, radius: float):
        self.radius = radius

    def calculate_perimeter(self) -> float:
        return 2 * self.PI * self.radius

show(Circle(radius=1)) # -> AttributeError: 'Circle' object has no attribute 'calculate_area'

In [168]:
check_last()

temp.txt:25: error: Argument 1 to "show" has incompatible type "Circle"; expected "Figure"
temp.txt:25: note: "Circle" is missing following "Figure" protocol member:
temp.txt:25: note:     calculate_area
Found 1 error in 1 file (checked 1 source file)


### `Неявные и явные реализации протокола`

- Явная реализация соответствует номинальной типизации
- Неявная - структурной

Явная типизация аналогична работе с абстрактными классами, можно использовать реализации методов по-умолчанию:

In [174]:
from abc import abstractmethod
from typing import Protocol

class Readable(Protocol):
    @abstractmethod
    def read(self) -> str: pass

    def get_size(self) -> int: return 1000

class File(Readable):
    def read(self) -> str: return 'file content'

print(File().get_size())  # Выведет 1000

class WrongFile(Readable):
    def read(self) -> int: return 42

1000


In [175]:
check_last()

temp.txt:16: error: Return type "int" of "read" incompatible with return type "str" in supertype "Readable"
Found 1 error in 1 file (checked 1 source file)


### `Forward references`

- Иногда в коде возникает необходимость сослаться на тип, который ещё не был определён


__Примеры:__

In [ ]:
class Foo:
    def bar(self) -> Foo:
        return Foo()

# NameError: name 'Foo' is not defined

In [ ]:
class Foo:
    def bar(self) -> Bar:
        return Bar()

class Bar:
    def foo(self) -> Foo:
        return Foo()

# NameError: name 'Bar' is not defined

### `Forward references`

- Решение проблемы является использования строкового представления имени типа вместо самого типа:

In [202]:
class Foo:
    def bar(self) -> 'Foo':
        return Foo()

In [203]:
class Foo:
    def bar(self) -> 'Bar':
        return Bar()

class Bar:
    def foo(self) -> Foo:
        return Foo()

- Начиная с Python 3.7 заботу об этом может взять на себя импорт `from __future__ import annotations`
- Он автоматически производит замену всех типов на имена-строки

In [204]:
from __future__ import annotations

class Foo:
    def bar(self) -> Foo:
        return Foo()

### `Хранение типизированных данных`

- Хранение типизированных данных требует строгости и аккуратности
- В простейших случаях можно воспользоваться словарём или кортежем:

In [ ]:
tuple_data = (0, 'string')
dict_data = {'int_field': 0, 'str_field': 'string'}

- отсутствие именованного типа может привести к ошибке, нужно помнить, что переменная ссылается на нужную стркутуру
- необходимо следить за ключами словаря и порядком аргументов кортежа, это автоматически не проверяется


- Возможный вариант - _именованные кортежи_ (Namedtuple):

In [214]:
from collections import namedtuple

Data = namedtuple('Data', ['int_field', 'str_field'])
named_tuple_data = Data(0, 'string')

- Проблемы этого подхода:
    - кортежи являются неизменяемыми
    - по кортежам можно итерироваться
    - кортежи с разными наборами ключей, но одинаковыми значениями будут считаться одинаковыми

In [219]:
Data2 = namedtuple('Data2', ['int_field_2', 'str_field_2'])

print(*named_tuple_data)
print(Data2(0, 'string') == named_tuple_data)

0 string
True


### `Классы данных`

- Эти проблемы решаются созданием для данных отдельного типа


- Пример типичного класса данных, описанного стандартными средствами языка:

In [211]:
class Data:
    def __init__(self, int_field: int, str_field: str):
        self.int_field = int_field
        self.str_field = str_field

    def __repr__(self):
        return f"{self.__class__.__name__}(int_field={self.int_field:d}, str_field='{self.str_field:s}')"

    def __str__(self):
        return self.__repr__()

    def __eq__(self, other):
        return self.int_field == other.int_field and self.str_field == other.str_field

- Многие вещи выглядят типовыми и повторяющимися

### `Классы данных`

- Для решения проблем хранения типизированных данных в Python 3.7 были добавлены _классы данных_
- Этот механизм позволяет автоматически генерировать классы с типизированными полями со значениями по-умолчанию
- Пример того же класса данных, описанного с помощью декоратора `@dataclass`:

In [213]:
from dataclasses import dataclass

@dataclass
class Data:
    int_field: int
    str_field: str

- Этот класс обладает аналогичной функциональностью
- Аннотации типов в `dataclass` обязательны, поля без типов будут проигнорированы


- Полям можно задавать значения по-умолчанию:

In [220]:
@dataclass
class Data:
    int_field: int = 10
    str_field: str = 'string'

Data()

Data(int_field=10, str_field='string')

### `Классы данных`

- Есть параметры для управление генерацией класса (все методы, определённые пользователем, перетирают реализации по-умолчанию):<br><br>
    - `frozen` - сделать класс неизменяемым или нет (по-умолчанию `False`)
    - `init` - сгенерировать для класса метод `__init__` (по-умолчанию `True`)
    - `repr` - сгенерировать для класса метод `__repr__` (по-умолчанию `True`)
    - `eq` - сгенерировать для класса метод `__eq__`, сраниваются типы и значения как кортежи (по-умолчанию `True`)
    - `order` - сгенерировать методы `__lt__`, `__le__`, `__gt__` и `__ge__`, сравнивая значения как кортежи (по-умолчанию `False`)
    - `unsafe_hash` - использовать небезопасные поля в подсчёте хэша объекта в методе `__hash__` (по-умолчанию `False`)

In [284]:
@dataclass(frozen=True, order=True)
class Data:
    int_field: int = 10
    str_field: str = 'string'

#Data().int_field = 5 -> FrozenInstanceError: cannot assign to field 'int_field'

Data(10) < Data(20)

True

- Классы данных можно преобразовывать в словари и кортежи:

In [286]:
from dataclasses import asdict, astuple

print(asdict(Data(20)))
print(astuple(Data(20)))

{'int_field': 20, 'str_field': 'string'}
(20, 'string')


### `Создание изменяемых полей`

- Создавать поля с изменяемыми типами и значениями по-умолчанию не так просто, как константные

In [ ]:
@dataclass
class Data:
    list_field: List[int] = []

# -> ValueError: mutable default <class 'list'> for field list_field is not allowed: use default_factory

- Корректное решение проблемы подсказывается в сообщении об ошибке:

In [231]:
from dataclasses import field

@dataclass
class Data:
    list_field: List[int] = field(default_factory=list)

### `Пост-инициализация`

- При создании объекта может потребоваться выполнить после `__init__` по-умолчанию некоторую логику, не переписывая весь метод
- Для этого можно определить метод `__post_init__`, который `__init__` (если он определён) всегда вызывает после себя
- Можно задавать параметры для пост-инициализации: это поля типа `InitVar`, которые передаются в `__post_init__`
- В остальном классе эти поля игнорируются

In [314]:
from dataclasses import InitVar

@dataclass
class Data:
    float_field: float
    int_field: InitVar[int]

    def __post_init__(self, int_field: int) -> None:
        print(f'Created Data object with float value {self.float_field:.{int_field}f}')

d = Data(3.141562, 2)
#d.int_field -> AttributeError: 'Data' object has no attribute 'int_field'

Created Data object with float value 3.14


### `Наследование классов данных`

- Классы данных наследуются от `object` и могут использоваться для получения новых классов путём наследования
- При этом наследуюемый класс тоже нужно помечать декоратором `@dataclass`
- Декоратор проходит по всем родительским классам (при множественном наследовании порядок MRO), для каждого класса поля сохраняются в упорядоченный словарь, остаются самые последние версии полей
- По этой причине, если поле в родительском классе определялось со значением по-умолчанию, то и в дочернем классе должно быть так (или останется старое значение)

In [282]:
@dataclass
class DataA:
    field_a: str = 'default'
    field_b: str = None

@dataclass
class DataB(DataA):
    field_a: int
    field_b: str
    field_c: int = None

#DataB.__init__ -> __init__(self, field_a: 'int' = 'default', field_b: 'str' = None, field_c: 'int' = None)

### `Типизированные словари TypedDict`

- Схожими возможностями обладают _типизированные словари_ (`TypedDict`)

In [318]:
from typing import TypedDict

class Data(TypedDict):
    int_field: int
    str_field: str

print(Data(int_field=10, str_field='string'))

{'int_field': 10, 'str_field': 'string'}


- Использование классов данных является предпочтительнее по нескольким причинам:<br><br>
    - классы поддерживают номинальную типизацию, а `TypedDict` - только структурную, т.е. `isinstance` использовать нельзя, нужно проверять соответствие полей во время выполнения кода, а это для больших структур может оказаться затратной процедурой<br><br>
    - классы удобнее с точки зрения предоставляемого интерфейса (например, наличие properties)

### `Сторонние альтернативы классам данных`

- Использование классов данных не требует дополнительных зависимостей, что иногда может быть важным<br><br>
- Но классы данных, как и прочие решения из стандартной библиотеки, не идеальны, например, не справляются с валидацией данных без написания громоздкого дополнительного кода<br><br>
- Есть более развитые сторонние библиотеки для работы с типизированными данными, наиболее популярны `attrs` и `pydantic`<br><br>
- На самом деле `attrs` появилась до Python 3.7, и классы данных создавались под её влиянием и с помощью её разработчиков<br><br>
- [Ссылка](https://stefan.sofa-rockers.org/2020/05/29/attrs-dataclasses-pydantic) на хорошую статью про сравнение классов данных и библиотек `attrs` и `pydantic`<br><br>

- Рассмотрим подробнее модуль `pydantic`, поскольку в связке с Fast API он особенно полезен при web-разработке<br><br>

### `Модуль pydantic`

- Создание объектов производится наследованием от базового класса:

In [326]:
from typing import Optional, List
from pydantic import BaseModel

class Data(BaseModel):
    int_field: int
    str_field: str
    list_field: Optional[List[str]]

Data(**{'int_field': 10, 'str_field': 'string'})

Data(int_field=10, str_field='string', list_field=None)

- Типы входных данных автоматически валидируются:

In [ ]:
Data(**{'int_field': 'string', 'str_field': 'string'})

#ValidationError: 1 validation error for Data
#int_field
#  value is not a valid integer (type=type_error.integer)

- И даже исправляются, если преобразование известно:

In [328]:
Data(**{'int_field': '10', 'str_field': 'string'})

Data(int_field=10, str_field='string', list_field=None)

### `Вложенные pydantic-классы`

- Легко создавать и использовать рекурсивные модели данных с утиной типизацией:

In [331]:
from typing import List
from pydantic import BaseModel

class Foo(BaseModel):
    count: int
    size: float = None

class Bar(BaseModel):
    apple = 'x'
    banana = 'y'

class Spam(BaseModel):
    foo: Foo
    bars: List[Bar]

m = Spam(foo={'count': 4}, bars=[{'apple': 'x1'}, {'apple': 'x2'}])

print(m, '\n')
print(m.dict())

foo=Foo(count=4, size=None) bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')] 

{'foo': {'count': 4, 'size': None}, 'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}]}


### `Валидация входных данных`

- Проверять можно не только типы, но значения входных данных:

In [ ]:
from pydantic import Field

class Data(BaseModel):
    str_field: str = Field(min_length=2, max_length=5)
    int_field: int = Field(le=150)

Data(str_field='', int_field=200)

#ValidationError: 2 validation errors for Data
#str_field
#  ensure this value has at least 2 characters (type=value_error.any_str.min_length; limit_value=2)
#int_field
#  ensure this value is less than or equal to 150 (type=value_error.number.not_le; limit_value=150)

- у типа `Field` есть много параметров для валидации, например:<br><br>
    - `regex` : проверка соответствия строки заданному регулярному выражению
    - `multiple_of` : проверка того, что целое число является множителем заданного числа
    - `max_items` / `min_items` : проверка числа элементов в параметра-коллекциях
    - `allow_mutation` : разрешение или запрет изменения содержимого поля

### `Валидация входных данных`

- Для более сложных случаев описываются методы валидации:

In [ ]:
from pydantic import BaseModel, validator
from typing import Optional

class Data(BaseModel):
    str_field: str
        
    @validator('str_field')
    def str_field_validator(cls, value):
        if len(value) != 10:
            raise ValueError('Phone number must have 10 digits')
        return value

Data(str_field='999999')
#ValidationError: 1 validation error for Data
#str_field
#  Phone number must have 10 digits (type=value_error)

### `Дополнительные типы`

- `pydanctic` предназначен для работы с реальными данными, поэтому в нём есть встроенные полезные типы:

In [348]:
from pydantic import (
    FilePath, HttpUrl, EmailStr, color,
    IPvAnyAddress, NegativeInt, PositiveFloat,
)

In [349]:
from pydantic import BaseModel, ValidationError

class Data(BaseModel):
    color: color.Color

print(Data(color='purple'))
print(Data(color='hsl(180, 100%, 50%)'))
print(Data(color='hsl(179, 100%, 50%)'))

#print(Data(color='hello')) ->

#ValidationError: 1 validation error for Data
#color
#  value is not a valid color: string not recognised as a valid color ...

color=Color('purple', rgb=(128, 0, 128))
color=Color('cyan', rgb=(0, 255, 255))
color=Color('#00fffb', rgb=(0, 255, 251))


### `Парсинг переменных окружения`

- `pydantic` позволяет парсить данные из файлов типа `.env` и напрямую приводить их к объектам типа `BaseSettings `
- Для этого нужно дополнительно установить модуль `python-dotenv`

In [2]:
with open('.env', 'w') as fout:
    fout.write('''
        login=MelLain
        password=Password
    \n''')

In [3]:
from pydantic import BaseSettings

class Settings(BaseSettings):
    login: str
    password: str

    class Config:
        env_file = '.env'
        env_file_encoding = 'utf-8'

print(Settings())

login='MelLain' password='Password'


### `Генерация схемы данных`

In [5]:
from enum import Enum
from pydantic import BaseModel, Field

class Counter(BaseModel):
    count: int
    size: float = None

class Gender(str, Enum):
    male = 'male'
    female = 'female'
    other = 'other'
    not_given = 'not_given'

class Model(BaseModel):
    counter: Counter = Field(...)
    gender: Gender = Field(None, alias='Gender')
    snap: int = Field(42, title='The Snap', gt=30, lt=50)

    class Config:
        title = 'Main'

print(Model.schema_json(indent=2))

{
  "title": "Main",
  "type": "object",
  "properties": {
    "counter": {
      "$ref": "#/definitions/Counter"
    },
    "Gender": {
      "$ref": "#/definitions/Gender"
    },
    "snap": {
      "title": "The Snap",
      "default": 42,
      "exclusiveMinimum": 30,
      "exclusiveMaximum": 50,
      "type": "integer"
    }
  },
  "required": [
    "counter"
  ],
  "definitions": {
    "Counter": {
      "title": "Counter",
      "type": "object",
      "properties": {
        "count": {
          "title": "Count",
          "type": "integer"
        },
        "size": {
          "title": "Size",
          "type": "number"
        }
      },
      "required": [
        "count"
      ]
    },
    "Gender": {
      "title": "Gender",
      "description": "An enumeration.",
      "enum": [
        "male",
        "female",
        "other",
        "not_given"
      ],
      "type": "string"
    }
  }
}


### `Спасибо за внимание!`